In [4]:
from html import escape

def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f"{a}(<i>{hex(a)}</i>)"

def html_real(a):
    return f"{round(a, 2):.2f}"

def html_str(s):
    return html_escape(s).replace("\n", "<br/>\n")

def html_list(l):
    items = (f"<li>{html_escape(item)}</li>" for item in l)
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

def html_dict(d):
    items = (f"<li>{k}={v}</li>" for k, v in d.items())
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

In [5]:
print(html_str("""this is a multi line
string with many 
special characters: 10 < 100"""))

this is a multi line<br/>
string with many <br/>
special characters: 10 &lt; 100


In [6]:
print(html_int(255))

255(<i>0xff</i>)


In [7]:
print(html_escape(3+10j))

(3+10j)


In [8]:
from decimal import Decimal

def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        return html_escape(arg)


    

In [9]:
htmlize("""Python rocks""")

'Python rocks'

In [10]:
print(htmlize([1, 2, 3]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
</ul>


In [3]:
from decimal import Decimal
from html import escape

def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f"{a}(<i>{hex(a)}</i>)"

def html_real(a):
    return f"{round(a, 2):.2f}"

def html_str(s):
    return html_escape(s).replace("\n", "<br/>\n")

def html_list(l):
    items = (f"<li>{htmlize(item)}</li>" for item in l)
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

def html_dict(d):
    items = (f"<li>{html_escape(k)}={htmlize(v)}</li>" for k, v in d.items())
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        return html_escape(arg)

In [5]:
print(htmlize(["""Python rocks 0 < 1
               """, (1, 2, 3), 100]))

<ul>
<li>Python rocks 0 &lt; 1<br/>
               </li>
<li><ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul></li>
<li>100(<i>0x64</i>)</li>
</ul>


In [6]:
def htmlize(arg):
    registry = {
        object: html_escape,
        int: html_int,
        float: html_int,
        Decimal: html_int,
        str: html_str,
        list: html_list,
        tuple: html_list,
        dict: html_dict
    }
    
    fn = registry.get(type(arg), registry[object])
    return fn(arg)

In [7]:
htmlize(100)

'100(<i>0x64</i>)'

In [8]:
def singledispatch(fn):
    registry = {}
    registry[object] = fn
    
    def inner(arg):
        return registry[object](arg)
        
    return inner

In [9]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [12]:
def singledispatch(fn):
    registry = {}
    registry[object] = fn
    registry[int] = lambda a: f"{a}(<i>{hex(a)}</i>)"
    registry[str] = lambda s: escape(s).replace("\n", "<br/>\n")
    
    
    def inner(arg):
        f = registry.get(type(arg), registry[object])
        return f(arg)
        
    return inner

In [13]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [14]:
htmlize(100)

'100(<i>0x64</i>)'

In [47]:
def singledispatch(fn):
    registry = {}
    registry[object] = fn

    def decorated(arg):
        return registry.get(type(arg), registry[object])(arg)

    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    decorated.register = register
    decorated.registry = registry
    return decorated

In [48]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [49]:
htmlize

<function __main__.singledispatch.<locals>.decorated(arg)>

In [50]:
htmlize("1 < 100")

'1 &lt; 100'

In [51]:
htmlize.register(int)(lambda a: f"{a}(<i>{hex(a)}</i>)")

<function __main__.<lambda>(a)>

In [52]:
htmlize(100)

'100(<i>0x64</i>)'

In [53]:
@htmlize.register(int)
def html_int(a):
    return f"{a}<{hex(a)}>"

In [54]:
htmlize(100)

'100<0x64>'

In [55]:
html_int

<function __main__.html_int(a)>

In [56]:
@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l):
    items = (f"<li>{htmlize(item)}</li>" for item in l)
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

In [57]:
print(htmlize((1, 2, 3)))

<ul>
<li>1<0x1></li>
<li>2<0x2></li>
<li>3<0x3></li>
</ul>


In [58]:
htmlize.registry

{object: <function __main__.htmlize(a)>,
 int: <function __main__.html_int(a)>,
 list: <function __main__.html_sequence(l)>,
 tuple: <function __main__.html_sequence(l)>}